In [1]:
import polars as pl
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

import sys
work_path = Path(next((p for p in Path("__file__").resolve().parents if p.name == "Research"), None))
torchlib_path = str(work_path / Path("app/torch_libs"))
sys.path.append(torchlib_path)

from run_manager import RunManager, RunsManager, RunViewer


In [2]:
nb_path = Path().resolve()
# rm = RunManager(exp_path=nb_path)
# rm.ref_stats()
rv = RunViewer(exp_path=nb_path)
df_base = rv.fetch_results()

pass

In [11]:
df_stats = rv.fetch_results().cast({"run_id": pl.Int32})
df_mets = rv.fetch_metrics()
df_mets = df_mets.join(df_stats, on="run_id", how="left")

display(df_mets)


run_id,step,lr,epoch,train_loss,train_acc,val_loss,val_acc,model_arc,train_dataset,val_dataset,num_classes,train_trans,val_trans,train_num,val_num,epochs,max_lr,batch_size,iters/epoch,base_fils,fils,ensembles,classdist,classdist2,classdist3,params,loss_func,optimizer,scheduler,lr_right,epoch_right,train_loss_right,train_acc_right,val_loss_right,val_acc_right
i32,i64,f64,i64,f64,f64,f64,f64,str,str,str,i64,str,str,i64,i64,i64,f64,i64,i64,i64,i64,i64,f64,f64,f64,i64,str,str,str,f64,i64,f64,f64,f64,f64
102,1,0.0005,1,0.353967,0.326,null,null,"""models.gitresnet_ee resnet18""","""cifar100_train""","""cifar100_val""",10,"""[ToTensor(), RandomHorizontalF…","""[ToTensor()]""",5000,1000,200,0.0005,128,40,32,1,1024,1.617023,1.1595737,0.6382765,3062784,"""CrossEntropyLoss()""","""Adam ( Parameter Group 0 a…","""CosineAnnealingLR ( T_max = 20…",3.0842e-8,200,0.15679,0.6932,0.190502,0.578
102,2,0.0005,2,0.326281,0.3662,0.351518,0.19,"""models.gitresnet_ee resnet18""","""cifar100_train""","""cifar100_val""",10,"""[ToTensor(), RandomHorizontalF…","""[ToTensor()]""",5000,1000,200,0.0005,128,40,32,1,1024,1.617023,1.1595737,0.6382765,3062784,"""CrossEntropyLoss()""","""Adam ( Parameter Group 0 a…","""CosineAnnealingLR ( T_max = 20…",3.0842e-8,200,0.15679,0.6932,0.190502,0.578
102,3,0.0005,3,0.312804,0.3828,null,null,"""models.gitresnet_ee resnet18""","""cifar100_train""","""cifar100_val""",10,"""[ToTensor(), RandomHorizontalF…","""[ToTensor()]""",5000,1000,200,0.0005,128,40,32,1,1024,1.617023,1.1595737,0.6382765,3062784,"""CrossEntropyLoss()""","""Adam ( Parameter Group 0 a…","""CosineAnnealingLR ( T_max = 20…",3.0842e-8,200,0.15679,0.6932,0.190502,0.578
102,4,0.0005,4,0.303887,0.3954,0.294532,0.393,"""models.gitresnet_ee resnet18""","""cifar100_train""","""cifar100_val""",10,"""[ToTensor(), RandomHorizontalF…","""[ToTensor()]""",5000,1000,200,0.0005,128,40,32,1,1024,1.617023,1.1595737,0.6382765,3062784,"""CrossEntropyLoss()""","""Adam ( Parameter Group 0 a…","""CosineAnnealingLR ( T_max = 20…",3.0842e-8,200,0.15679,0.6932,0.190502,0.578
102,5,0.0005,5,0.295542,0.4088,null,null,"""models.gitresnet_ee resnet18""","""cifar100_train""","""cifar100_val""",10,"""[ToTensor(), RandomHorizontalF…","""[ToTensor()]""",5000,1000,200,0.0005,128,40,32,1,1024,1.617023,1.1595737,0.6382765,3062784,"""CrossEntropyLoss()""","""Adam ( Parameter Group 0 a…","""CosineAnnealingLR ( T_max = 20…",3.0842e-8,200,0.15679,0.6932,0.190502,0.578
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
425,156,0.00006,156,0.025383,0.95,0.245998,0.653,"""models.gitresnet_ee resnet18""","""cifar100_train""","""cifar100_val""",10,"""[ToTensor(), RandomHorizontalF…","""[ToTensor()]""",5000,1000,200,0.0005,128,40,32,32,1,3.581213,1.6257433,0.479573,2797610,"""CrossEntropyLoss()""","""Adam ( Parameter Group 0 a…","""CosineAnnealingLR ( T_max = 20…",0.00005,160,0.024056,0.9538,0.252723,0.642
425,157,0.000057,157,0.0256,0.9528,null,null,"""models.gitresnet_ee resnet18""","""cifar100_train""","""cifar100_val""",10,"""[ToTensor(), RandomHorizontalF…","""[ToTensor()]""",5000,1000,200,0.0005,128,40,32,32,1,3.581213,1.6257433,0.479573,2797610,"""CrossEntropyLoss()""","""Adam ( Parameter Group 0 a…","""CosineAnnealingLR ( T_max = 20…",0.00005,160,0.024056,0.9538,0.252723,0.642
425,158,0.000055,158,0.025231,0.9506,0.251119,0.649,"""models.gitresnet_ee resnet18""","""cifar100_train""","""cifar100_val""",10,"""[ToTensor(), RandomHorizontalF…","""[ToTensor()]""",5000,1000,200,0.0005,128,40,32,32,1,3.581213,1.6257433,0.479573,2797610,"""CrossEntropyLoss()""","""Adam ( Parameter Group 0 a…","""CosineAnnealingLR ( T_max = 20…",0.00005,160,0.024056,0.9538,0.252723,0.642


In [5]:
def rv_to_figs(rv, place_row_col, place_col_col, graph_x_col, graph_y_col, label_col, window_size=None):
    df_stats = rv.fetch_results().cast({"run_id": pl.Int32})
    df_mets = rv.fetch_metrics()

    df_mets = df_mets.join(df_stats, on="run_id", how="left").sort(by=place_row_col).sort(by=place_col_col)

    row_vals = df_stats[place_row_col].unique().reverse()
    col_vals = df_stats[place_col_col].unique()

    fig, ax = plt.subplots(nrows=len(row_vals), ncols=len(col_vals), squeeze=False)
    plt.subplots_adjust(hspace=0.35)
    fig.set_figwidth(6 * len(col_vals))
    fig.set_figheight(4 * len(row_vals))

    for r, row_val in enumerate(row_vals):
        for c, col_val in enumerate(col_vals):

            df_stats_ext = df_stats.filter((pl.col(place_row_col) == row_val) & (pl.col(place_col_col) == col_val))
            run_ids = df_stats_ext["run_id"]
            

            df_agg = df_mets.group_by(by="run_id", maintain_order=True).agg(pl.all())
            df_agg = df_agg.filter((pl.col("run_id").is_in(run_ids)))


            ax[r][c].set_xlabel(graph_x_col)
            ax[r][c].set_ylabel(graph_y_col)
            ax[r][c].set_ylim([0, 1])
            ax[r][c].set_title(f"{place_row_col}: {row_val}, {place_col_col}: {col_val}")

            for df_row in df_agg.iter_rows(named=True):
                x = pl.Series(df_row[graph_x_col])
                y = pl.Series(df_row[graph_y_col]).rolling_mean(window_size=window_size, min_periods=0)
                # label = df_stats.filter(pl.col("run_id") == df_row["run_id"])[label_col][0]
                # ax[r][c].plot(x, y, label=label)

                # ax[r][c].plot(x, y)
                # y2 = pl.Series(df_row["train_acc"]).rolling_mean(window_size=window_size, min_periods=0)
                # ax[r][c].plot(x, y2)

                ax[r][c].plot(x, y)
                y2 = pl.Series(df_row["train_acc"]).rolling_mean(window_size=window_size, min_periods=0)
                ax[r][c].plot(x, y2)

                # ax[r][c].legend()


rv_to_figs(rv, "fils", "num_data", "epoch", "val_acc", "num_data", window_size=4)


ColumnNotFoundError: "num_data" not found

In [3]:
def heat_pivot(df, values, index, columns, aggregate_function=None, cmap=None, xlabels=None, ylabels=None, annot_rescale=False, color_rescale=False):
    df = df.sort(columns)
    df = df.pivot(values=values, index=index, columns=columns, aggregate_function=aggregate_function)
    df = df.reverse()
    # df = df.select(df.columns[::-1])
    df_ind_ser= df.select(index).to_series()
    df = df.select(pl.all().exclude(index))

    if xlabels is None:
        xlabels = df.columns
    if ylabels is None:
        ylabels = df_ind_ser
    
    
    # heatmap用
    def rescale(df):
        for col_name in df.columns:
            col_ser = df[col_name]
            max_val = col_ser.max()
            min_val = col_ser.min()
            
            if max_val != min_val:
                df = df.with_columns(pl.col(col_name).sub(min_val).truediv(max_val - min_val))
        return df
    re_df = rescale(df)
    

    if annot_rescale:
        annot_C = re_df.to_numpy()
    else:
        annot_C = df.to_numpy()

    if color_rescale:
        color_C = re_df.to_numpy()
    else:
        color_C = df.to_numpy()
    
    annot_C *= 100


    sns.set_theme(rc={"figure.figsize": (len(xlabels)/1.5, len(ylabels)/1.5)})
    ax = sns.heatmap(color_C, annot=annot_C, cbar=True, square=True, cmap=cmap, xticklabels=xlabels, yticklabels=ylabels, fmt=".1f")
    ax.set_xlabel("# of data", fontsize=16)
    ax.set_ylabel("(fils, ensembles)", fontsize=14, rotation=90)
    plt.show()
    
    

df_stats = rv.fetch_results()
df_stats_ext = df_stats

df_stats_ext = df_stats_ext.with_columns((pl.col("fils") * pl.col("fils") * pl.col("ensembles")).alias("fi_en"))
df_stats_ext = df_stats_ext.with_columns(pl.concat_str([pl.lit("("), pl.col("fils"), pl.lit(","), pl.col("ensembles"), pl.lit(")")]).alias("fe"))

for fi_en in df_stats_ext["fi_en"].unique().sort():
    df_stats_fi = df_stats_ext.filter(pl.col("fi_en") == fi_en)
    # df_stats_ext = df_stats_ext.sort("fils")

    heat_pivot(df_stats_fi, values="val_acc", index="fe", columns="num_data", cmap="Blues_r", aggregate_function="mean", annot_rescale=False, color_rescale=True)




ColumnNotFoundError: "num_data" not found